# ReAct

This walkthrough showcases using an agent to implement the [ReAct](https://react-lm.github.io/) logic.

In [19]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain_community.llms import OpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub

## Load tools

Let's load some tools to use.

In [4]:
tools = [TavilySearchResults(max_results=1)]

## Create Agent

In [21]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

In [22]:
print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


In [23]:
# Choose the LLM to use
llm = OpenAI()

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

## Run the Agent

In [24]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [25]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
 I should search for this term
Action: tavily_search_results_json
Action Input: LangChain[{'url': 'https://www.ibm.com/topics/langchain', 'content': "LangChain is essentially a library of abstractions for Python and Javascript, representing common steps and concepts  LangChain is an open source orchestration framework for the development of applications using large language models  other LangChain features, like the eponymous chains.  LangChain provides integrations for over 25 different embedding methods, as well as for over 50 different vector storesLangChain is an open source orchestration framework for the development of applications using large language models (LLMs). Available in both Python- and Javascript-based libraries, LangChain's tools and APIs simplify the process of building LLM-driven applications like chatbots and virtual agents . LangChain serves as a generic interface for ..."}] I should read the content of the first result
Acti

{'input': 'what is LangChain?',
 'output': 'LangChain is an open source orchestration framework for the development of applications using large language models. It provides integrations for over 25 different embedding methods and over 50 different vector stores. It simplifies the process of building LLM-driven applications like chatbots and virtual agents.'}

## Using LCEL

We will first show how to create the agent using LCEL

In [26]:
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

In [27]:
prompt = hub.pull("hwchase17/react")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

In [28]:
llm_with_stop = llm.bind(stop=["\nObservation"])

In [29]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

In [30]:
from langchain.agents import AgentExecutor

In [31]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [32]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
 As a researcher, I should always think about using reliable sources to find information.
Action: tavily_search_results_json
Action Input: "LangChain"[{'url': 'https://www.ibm.com/topics/langchain', 'content': "LangChain is essentially a library of abstractions for Python and Javascript, representing common steps and concepts  LangChain is an open source orchestration framework for the development of applications using large language models  other LangChain features, like the eponymous chains.  LangChain provides integrations for over 25 different embedding methods, as well as for over 50 different vector storesLangChain is an open source orchestration framework for the development of applications using large language models (LLMs). Available in both Python- and Javascript-based libraries, LangChain's tools and APIs simplify the process of building LLM-driven applications like chatbots and virtual agents . LangChain serves as a generic interface 

{'input': 'what is LangChain?',
 'output': 'LangChain is an open source orchestration framework for developing applications using large language models. It provides tools and APIs for simplifying the process of building LLM-driven applications, such as chatbots and virtual agents. It also offers integrations for various embedding methods and vector stores.'}

## Using chat models

You can also create ReAct agents that use chat models instead of LLMs as the agent driver.

The main difference here is a different prompt. We will use JSON to encode the agent's actions (chat models are a bit tougher to steet, so using JSON helps to enforce the output format).

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
chat_model = ChatOpenAI(temperature=0)

In [ ]:
prompt = hub.pull("hwchase17/react-json")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

In [ ]:
chat_model_with_stop = chat_model.bind(stop=["\nObservation"])

In [ ]:
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser

In [ ]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke(
    {
        "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
    }
)